In [1]:
require 'cutorch'
require 'nn'
require 'cunn'

In [2]:
-- load data file
dataFile = '../data/hh.t7'
hh = torch.load(dataFile)

In [3]:
trainset = {}
train_pos = hh.train.labels:nonzero()
train_neg = torch.add(hh.train.labels,-1):nonzero()

-- create training set with every other example a negative
neg_stride = math.floor(train_neg:size(1) / train_pos:size(1))
numTrainPairs = train_pos:size(1)
numTrainExamples = numTrainPairs*2
trainData = torch.Tensor(numTrainExamples,1,240,320)
trainLabels = torch.Tensor(numTrainExamples)
for i = 1,numTrainPairs do
    if i % 10 == 0 then
        print('processing training pair ' .. i)
    end

    local pos_idx = train_pos[i]
    local neg_idx = train_neg[(i-1)*neg_stride+1]

    trainData[2*i-1] = hh.train.data:narrow(1,pos_idx[1],1):float()
    trainLabels[2*i-1] = hh.train.labels:narrow(1,pos_idx[1],1) + 1

    trainData[2*i] = hh.train.data:narrow(1,neg_idx[1],1):float()
    trainLabels[2*i] = hh.train.labels:narrow(1,neg_idx[1],1) + 1
end

trainset.data = trainData
trainset.labels = trainLabels

print(trainset)
    
testset = hh.test

processing training pair 10	


processing training pair 20	
processing training pair 30	


processing training pair 40	


processing training pair 50	


processing training pair 60	


...

processing training pair 170	


processing training pair 180	


processing training pair 190	


processing training pair 200	


{
  data : DoubleTensor - size: 418x1x240x320
  labels : DoubleTensor - size: 418
}


In [4]:
setmetatable(trainset, 
    {__index = function(t, i) 
                    return {t.data[i], t.labels[i]} 
                end}
);
trainset.data = trainset.data:double() -- convert the data from a ByteTensor to a DoubleTensor.

function trainset:size() 
    return self.data:size(1) 
end

setmetatable(testset, 
    {__index = function(t, i) 
                    return {t.data[i], t.labels[i]} 
                end}
);
testset.data = testset.data:double() -- convert the data from a ByteTensor to a DoubleTensor.

function testset:size() 
    return self.data:size(1) 
end

In [5]:
mean = {} -- store the mean, to normalize the test set in the future
stdv  = {} -- store the standard-deviation for the future
for i=1,1 do -- over each image channel
    mean[i] = trainset.data[{ {}, {i}, {}, {}  }]:mean() -- mean estimation
    print('Channel ' .. i .. ', Mean: ' .. mean[i])
    trainset.data[{ {}, {i}, {}, {}  }]:add(-mean[i]) -- mean subtraction
    
    stdv[i] = trainset.data[{ {}, {i}, {}, {}  }]:std() -- std estimation
    print('Channel ' .. i .. ', Standard Deviation: ' .. stdv[i])
    trainset.data[{ {}, {i}, {}, {}  }]:div(stdv[i]) -- std scaling
end

-- view an image
-- itorch.image(trainset.data[301])

Channel 1, Mean: 0.21585217551476	


Channel 1, Standard Deviation: 0.069773298737423	


In [7]:
-- define network
net = nn.Sequential()
net:add(nn.SpatialConvolution(1, 6, 7, 7, 2, 2))
net:add(nn.SpatialMaxPooling(3,3,3,3))  
net:add(nn.SpatialConvolution(6, 16, 5, 5, 2, 2))
net:add(nn.SpatialMaxPooling(2,2,2,2))
-- net:add(nn.SpatialConvolution(16, 16, 5, 5))
-- net:add(nn.SpatialMaxPooling(2,2,2,2))
net = net:cuda()
-- out = net:forward(trainset.data[1]) -- must cuda the data first
-- print(out:size())
    
net:add(nn.View(16*9*12))                   
net:add(nn.Linear(16*9*12, 50))            
net:add(nn.Linear(50, 2)) 
--net:add(nn.Linear(120, 84))
--net:add(nn.Linear(84, 2))                  
net:add(nn.LogSoftMax())                    

criterion = nn.ClassNLLCriterion()

In [8]:
--train
net = net:cuda()
criterion = criterion:cuda()
trainset.data = trainset.data:cuda()

trainer = nn.StochasticGradient(net, criterion)
trainer.learningRate = 0.001
trainer.maxIteration = 10

trainer:train(trainset)

# StochasticGradient: training	


# current error = 0.49706102921917	


# current error = 0.36690123464787	


# current error = 0.3006043794908	


# current error = 0.24953644918768	


...

# current error = 0.17740567042782	


# current error = 0.1506308439103	


# current error = 0.1301967649939	


# current error = 0.11524672014862	


# current error = 0.10315171069506	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.10315171069506	


In [9]:
-- test accuracy

testset.data = testset.data:double()   -- convert from Byte tensor to Double tensor
for i=1,1 do -- over each image channel
    testset.data[{ {}, {i}, {}, {}  }]:add(-mean[i]) -- mean subtraction    
    testset.data[{ {}, {i}, {}, {}  }]:div(stdv[i]) -- std scaling
end
testset.data = testset.data:cuda()

testSize = #testset.data
numTestExamples = testSize[1]
correct = 0
for i=1,numTestExamples do
    local groundtruth = testset.labels[i] + 1
    local prediction = net:forward(testset.data[i])
    local confidences, indices = torch.sort(prediction, true)  -- true means sort in descending order
    if groundtruth == indices[1] then
        correct = correct + 1
    end
end

print(correct, 100*correct/numTestExamples .. ' % ')



3149	61.324245374878 % 	


In [10]:
-- test accuracy by class

class_preds = {0, 0}
class_performance = {0, 0}
class_counts = {0, 0}
for i=1,numTestExamples do
    local groundtruth = testset.labels[i] + 1
    class_counts[groundtruth] = class_counts[groundtruth] + 1
    local prediction = net:forward(testset.data[i])
    local confidences, indices = torch.sort(prediction, true)  -- true means sort in descending order
    class_preds[indices[1]] = class_preds[indices[1]] + 1
    if groundtruth == indices[1] then
        class_performance[groundtruth] = class_performance[groundtruth] + 1
    end
end

for i=1,2 do
    print('accuracy ' .. i .. ',' .. 100*class_performance[i]/class_counts[i] .. ' %')
end

print(class_preds)
print(class_performance)
print(class_counts)

accuracy 1,56.531845653185 %	
accuracy 2,86.074429771909 %	
{
  1 : 2548
  2 : 2587
}
{
  1 : 2432
  2 : 717
}
{
  1 : 4302
  2 : 833
}


In [11]:
-- train accuracy

trainSize = #trainset.data
numTrainExamples = trainSize[1]
correct = 0
for i=1,numTrainExamples do
    local groundtruth = trainset.labels[i]
    local prediction = net:forward(trainset.data[i])
    local confidences, indices = torch.sort(prediction, true)  -- true means sort in descending order
    if groundtruth == indices[1] then
        correct = correct + 1
    end
end

print(correct, 100*correct/numTrainExamples .. ' % ')

408	97.607655502392 % 	


In [12]:
-- train accuracy by class

class_performance = {0, 0}
class_counts = {0, 0}
class_preds = {0,0}
for i=1,trainset:size() do
    local groundtruth = trainset.labels[i]
    class_counts[groundtruth] = class_counts[groundtruth] + 1
    local prediction = net:forward(trainset.data[i])
    local confidences, indices = torch.sort(prediction, true)  -- true means sort in descending order
    class_preds[indices[1]] = class_preds[indices[1]] + 1
    if groundtruth == indices[1] then
        class_performance[groundtruth] = class_performance[groundtruth] + 1
    end
end

for i=1,2 do
    print('accuracy ' .. i .. ',' .. 100*class_performance[i]/class_counts[i] .. ' %')
end
print(class_preds)
print(class_performance)
print(class_counts)

accuracy 1,96.172248803828 %	
accuracy 2,99.043062200957 %	
{
  1 : 203
  2 : 215
}
{
  1 : 201
  2 : 207
}
{
  1 : 209
  2 : 209
}
